In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import json
import pandas as pd
from tqdm import tqdm

In [5]:
root = "drive/MyDrive/data/bank/"

In [4]:
with open(root + "data.json","r",encoding="utf8") as jf:
    stocks = json.load(jf)

NameError: ignored